In [14]:
import google.generativeai as genai
import os
from google.colab import userdata
import json

# Configure API key and initialize model
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('api_key')

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

def detect_booking_topic(input_text):
    """
    Detects whether the input text is about Room Booking, Vehicle Booking, or Leave Management.
    """

    model = genai.GenerativeModel("gemini-1.5-pro")

    prompt = f"""
    Classify the following input into one of these categories: Room Booking, Vehicle Booking, or Leave Management.

    Input: "{input_text}"
    Output the category name only.
    """

    response = model.generate_content(prompt)
    return response.text.strip()

def extract_entities(input_text, topic):
    """
    Extracts relevant fields from user input based on the detected topic.
    """
    model = genai.GenerativeModel("gemini-1.5-pro")

    field_templates = {
        "Room Booking": "room_name, start_date, end_date, start_time, end_time, number_of_participants, purpose",
        "Vehicle Booking": "pickup_place, destination, date, start_time, end_time, number_of_participants",
        "Leave Management": "start_date, end_date, start_portion, end_portion, purpose"
    }

    prompt = f"""
    Extract the following fields from the input text if present: {', '.join(field_templates[topic])}.
    Return a JSON object with only detected fields in this format:
    {{"field_name": "value"}}

    Input: "{input_text}"
    """

    response = model.generate_content(prompt)
    try:
        extracted_data = json.loads(response.text.strip())
        if not isinstance(extracted_data, dict):
            raise ValueError("Invalid JSON format")
    except (json.JSONDecodeError, ValueError):
        extracted_data = {}

    return extracted_data

def main():
    required_fields = {
        "Room Booking": ["room_name", "start_date", "end_date", "start_time", "end_time", "number_of_participants", "purpose"],
        "Vehicle Booking": ["pickup_place", "destination", "date", "start_time", "end_time", "number_of_participants"],
        "Leave Management": ["start_date", "end_date", "start_portion", "end_portion", "purpose"]
    }

    while True:
      user_input = input("Enter your request: ")
      topic = detect_booking_topic(user_input)

      if topic not in required_fields:
          print("Sorry, I couldn't determine the topic. Please try again.")
          continue

      session_data = extract_entities(user_input, topic)

      # Ask for missing fields only
      for field in required_fields[topic]:
            if field not in session_data:
                session_data[field] = input(f"Please provide {field.replace('_', ' ')}: ")

      print(f"Your {topic.lower()} request is complete: {json.dumps(session_data, indent=2)}")

      new_query = input("Would you like to make another booking? (yes/no): ").strip().lower()
      if new_query != "yes":
          print("Thank you for using the booking system!")
          break

if __name__ == "__main__":
    main()


Enter your request: Book a space for meeting
Please provide room name: Varendra
Please provide start date: 4 May
Please provide end date: 7 May
Please provide start time: 5 pm
Please provide end time: 6 pm
Please provide number of participants: 78
Please provide purpose: Meeting
Your room booking request is complete: {
  "room_name": "Varendra",
  "start_date": "4 May",
  "end_date": "7 May",
  "start_time": "5 pm",
  "end_time": "6 pm",
  "number_of_participants": "78",
  "purpose": "Meeting"
}
Would you like to make another booking? (yes/no): yes
Enter your request: Need a leave
Please provide start date: tomorrow
Please provide end date: 20 March
Please provide start portion: 2nd half
Please provide end portion: 1st half
Please provide purpose: Personal emergency
Your leave management request is complete: {
  "start_date": "tomorrow",
  "end_date": "20 March",
  "start_portion": "2nd half",
  "end_portion": "1st half",
  "purpose": "Personal emergency"
}
Would you like to make anoth